<a href="https://colab.research.google.com/github/kmykprn/word-similarity/blob/main/SentenceBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install datasets evaluate transformers[sentencepiece,torch]
#! pip install accelerate -U #マルチGPUの場合速くなる
! pip install fugashi unidic-lite ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 26.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

modelName = "tohoku-nlp/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(modelName)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

In [4]:
text = "これはテストテキストです"

In [5]:
token = tokenizer.tokenize(text)
print(token)

['これ', 'は', 'テスト', 'テキスト', 'です']


In [6]:
from transformers import BertJapaneseTokenizer, BertModel
import torch


class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest",
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)


MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"  # <- v2です。
model = SentenceBertJapanese(MODEL_NAME)

sentences = ["暴走したAI", "暴走した人工知能"]
sentence_embeddings = model.encode(sentences, batch_size=8)

print("Sentence embeddings:", sentence_embeddings)


tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Sentence embeddings: tensor([[ 0.1376,  0.5348,  0.0560,  ...,  0.9439,  0.1351,  0.0652],
        [ 0.0604,  0.6181, -0.5663,  ...,  0.0606, -0.8503,  0.1538]])


In [7]:
sentences = ["お辞儀をしている男性会社員", "笑い袋", "テクニカルエバンジェリスト（女性）", "戦うAI", "笑う男性（5段階）",
"お金を見つめてニヤけている男性", "「ありがとう」と言っている人", "定年（女性）", "テクニカルエバンジェリスト（男性）", "スタンディングオベーション"]
sentence_vectors = model.encode(sentences)

In [10]:
print(len(sentence_vectors[0]))

768


In [13]:
import scipy.spatial

queries = ['暴走したAI', '暴走した人工知能', 'いらすとやさんに感謝', 'つづく']
query_embeddings = model.encode(queries).numpy()

closest_n = 5
for query, query_embedding in zip(queries, query_embeddings):
    # query_enbedingと, 他のセンテンスの比較
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_vectors, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(sentences[idx].strip(), "(Score: %.4f)" % (distance / 2))





Query: 暴走したAI

Top 5 most similar sentences in corpus:
戦うAI (Score: 0.2949)
スタンディングオベーション (Score: 0.3180)
笑い袋 (Score: 0.3394)
「ありがとう」と言っている人 (Score: 0.4213)
お金を見つめてニヤけている男性 (Score: 0.4636)




Query: 暴走した人工知能

Top 5 most similar sentences in corpus:
笑い袋 (Score: 0.3595)
スタンディングオベーション (Score: 0.3766)
笑う男性（5段階） (Score: 0.3917)
テクニカルエバンジェリスト（男性） (Score: 0.4051)
テクニカルエバンジェリスト（女性） (Score: 0.4321)




Query: いらすとやさんに感謝

Top 5 most similar sentences in corpus:
「ありがとう」と言っている人 (Score: 0.1586)
スタンディングオベーション (Score: 0.2181)
お辞儀をしている男性会社員 (Score: 0.3267)
笑い袋 (Score: 0.3441)
お金を見つめてニヤけている男性 (Score: 0.3575)




Query: つづく

Top 5 most similar sentences in corpus:
スタンディングオベーション (Score: 0.3406)
「ありがとう」と言っている人 (Score: 0.3459)
戦うAI (Score: 0.3808)
定年（女性） (Score: 0.4192)
お辞儀をしている男性会社員 (Score: 0.4359)


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

query = ['暴走したAI']
query_embedding= model.encode(query).numpy()

# query_enbedingと, 他のセンテンスの比較
similarities = cosine_similarity(query_embedding, sentence_vectors).flatten()
max_id = np.argmax(similarities)
print(max_id)
print(sentences[max_id])
print(similarities)

3
戦うAI
[0.05882588 0.32120496 0.04490201 0.41021872 0.04170368 0.07286242
 0.15742645 0.06666032 0.05021613 0.36408943]
